In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt
from lmfit import models
from bokeh.plotting import figure, output_notebook, show

/home/frederik/anaconda3/lib/python3.4/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
pc_henrik = '/home/arrow13/Git/FP/Roentgen/'
pc_frederik = '/home/frederik/Dokumente/FP/Roentgen'

In [3]:
var_pc = pc_frederik

In [7]:
cd $var_pc/Messdaten

/home/frederik/Dokumente/FP/Roentgen/Messdaten


In [308]:
data = np.loadtxt("Ge111_40_30_Ni.xy").T

In [309]:
plt.plot(data[0],data[1])
plt.show()

In [332]:
intervall_voigt = np.arange(np.where(data[0] > 27.16)[0][0], np.where(data[0] > 27.34)[0][0])
intervall_voigt_1 = np.arange(np.where(data[0] > 27.16)[0][0], np.where(data[0] > 27.2436)[0][0])
intervall_voigt_2 = np.arange(np.where(data[0] > 27.2436)[0][0], np.where(data[0] > 27.34)[0][0])
err = np.sqrt(data[1][intervall_voigt])
err = np.where(err == 0, 1, err)
err_1 = np.sqrt(data[1][intervall_voigt_1])
err_1 = np.where(err_1 == 0, 1, err_1)
err_2 = np.sqrt(data[1][intervall_voigt_2])
err_2 = np.where(err_2 == 0, 1, err_2)

In [333]:
voigt_1 = models.VoigtModel(prefix='voigt_1')
voigt_2 = models.VoigtModel(prefix='voigt_2')

pars_1 = voigt_1.guess(data[1][intervall_voigt_1],x=data[0][intervall_voigt_1])
pars_1.update(voigt_2.guess(data[1][intervall_voigt_2],x=data[0][intervall_voigt_2]))

pars_1['voigt_1gamma'].set(0.0001,vary=True,expr='')
pars_1['voigt_2gamma'].set(0.0001,vary=True,expr='')
pars_1['voigt_1amplitude'].set(1000)
pars_1['voigt_2amplitude'].set(1000)

model = voigt_1 + voigt_2

In [334]:
out = model.fit(data[1][intervall_voigt],pars_1,x=data[0][intervall_voigt],weights=1/(err))

In [335]:
print(out.fit_report())

[[Model]]
    (Model(voigt, prefix='voigt_1') + Model(voigt, prefix='voigt_2'))
[[Fit Statistics]]
    # function evals   = 110
    # data points      = 43
    # variables        = 8
    chi-square         = 2478.796
    reduced chi-square = 70.823
[[Variables]]
    voigt_1amplitude:   2321.31746 +/- 125.9522 (5.43%) (init= 1000)
    voigt_1sigma:       0.01781282 +/- 0.002014 (11.31%) (init= 0.0149357)
    voigt_1center:      27.2096465 +/- 0.000449 (0.00%) (init= 27.21076)
    voigt_1fwhm:        0.06414948 +/- 0.007253 (11.31%)  == '3.6013100*voigt_1sigma'
    voigt_1gamma:       0.00787155 +/- 0.003239 (41.15%) (init= 0.0001)
    voigt_2center:      27.2811167 +/- 0.000495 (0.00%) (init= 27.2776)
    voigt_2amplitude:   1864.08446 +/- 82.56106 (4.43%) (init= 1000)
    voigt_2sigma:       0.01197677 +/- 0.001439 (12.02%) (init= 0.0149357)
    voigt_2fwhm:        0.04313206 +/- 0.005185 (12.02%)  == '3.6013100*voigt_2sigma'
    voigt_2gamma:       0.01296469 +/- 0.001627 (12.56%) (in

In [336]:
out.plot()
plt.show()

In [337]:
from uncertainties import ufloat
from uncertainties.umath import log as lg
from uncertainties.umath import sin as si

In [338]:
hc = 12.39841875
d_si = 3.1355

In [339]:
E_1 = 8047.8
E_2 = 8027.8

In [340]:
def E(theta,n=1):
    return (n*hc)/(2*d_si*si((np.pi * theta)/180))

In [341]:
theta_1 = ufloat(out.best_values['voigt_1center']/2,0.0002)

In [342]:
theta_2 = ufloat(out.best_values['voigt_2center']/2,0.000274)

In [343]:
def d(E,theta):
    return (hc)/(2*E*si((np.pi * theta)/180))

In [344]:
d_1 = d(E_1,theta_1)

In [345]:
d_2 = d(E_2,theta_2)

In [346]:
d_1*1000

3.274745999619327+/-4.723277558243749e-05

In [347]:
d_2*1000

3.274466535173337+/-6.45273206477524e-05

In [352]:
(1-3.2660/(d_1*1000))*100

0.2670741370580765+/-0.001438481917685159

In [353]:
(1-3.2660/(d_2*1000))*100

0.25856227517955244+/-0.001965525579450339